<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/model1_cross_validation_Federica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
'''
@File name: model1.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import csv

import numpy as np
from numpy import savetxt

import pandas as pd
from pandas import DataFrame
from pandas import read_csv
from pandas import concat

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence

from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
from matplotlib.pyplot import legend

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
def generate_model(shape):
  
  model = Sequential()
  model.add(BatchNormalization())
 
  model.add(Dense(29, input_dim=shape, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  model.add(Dense(57, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  model.add(Dense(113, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  model.add(Dense(226, kernel_initializer='random_uniform',  bias_initializer='zeros'))
  model.add(LeakyReLU(alpha = 0.3))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [14]:
#LOAD DATA
print("Loading data...")

dataset = '/root/data/6_uHoo_featureDataset.arff'

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("Converting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

Loading data...
DATASET LOADED
Converting values...
CONVERSION DONE


In [15]:
#OPTIMIZERS
sgd = optimizers.SGD(lr=0.0001)
adm = optimizers.Adam(lr=0.0001)
ada = optimizers.Adadelta(lr=0.0001)
#rms = optimizers.RMSprop(lr=0.001)

#DEFINE 10-FOLD CROSS-VALIDATION
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
predictions = []
actual = []

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  model = generate_model(X[train].shape[-1])

  #COMPILE MODEL
  model.compile(loss='binary_crossentropy', optimizer = adm , metrics=['accuracy'])
  
  #EARLY STOPPING
  #es = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=0, mode='auto')

  #FIT MODEL
  history = model.fit(X[train], Y[train], epochs=45, validation_split=0.33, batch_size = 128, shuffle = True, verbose=0, class_weight = classWeight)#, callbacks = [es])

  #EVALUATE MODEL
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS AND SAVE IN A CSV FILE
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  predictions.append([i,pred])
  actual.append([i,Y[test]])
  #print(predictions[0])
  #print(actual[0])
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))



FOLD: 1
Test loss: 38.20%
Test accuracy: 80.84%

FOLD: 2
Test loss: 34.08%
Test accuracy: 85.60%

FOLD: 3
Test loss: 36.96%
Test accuracy: 82.94%

FOLD: 4
Test loss: 32.08%
Test accuracy: 86.69%

FOLD: 5
Test loss: 36.52%
Test accuracy: 83.70%

FOLD: 6
Test loss: 37.14%
Test accuracy: 82.65%

FOLD: 7
Test loss: 39.03%
Test accuracy: 81.44%

FOLD: 8
Test loss: 39.57%
Test accuracy: 81.31%

FOLD: 9
Test loss: 40.08%
Test accuracy: 80.68%

FOLD: 10
Test loss: 32.90%
Test accuracy: 85.84%
MEAN ACCURACY: 83.17% (STANDARD DEVIATION: +/- 2.10%)


Save model:

model.save('my_model1_cv.h5')

Load a saved medel:

model = load_model('my_model1_cv.h5')